# LIMITing your search

You may find yourself working with tables that contain so many records that simple queries can take forever to load. This is especially challenging when you're searching for the right table to use and just need a quick result. For these scenarios the trick is to `LIMIT` the number of rows that your query returns. In this exercise you will practice using this function.

```
SELECT * 
FROM payment
ORDER BY amount DESC
LIMIT 10;
```

# Which table to use?

Use the `SELECT * FROM ___ LIMIT 10` framework to find the table you need to answer the following question:

What categories of films does this company rent?

- `category`

# What tables are in your database?

You don't have to rely solely on knowing what tables exist. Instead, you can query the `pg_catalog.pg_tables` to list all of the tables that exist in your database.

Of course, this will list every table, including system tables so ideally, you want to limit your results to the schema where your data resides which in this case is `'public'`.

Note: This system table is specific to PostgreSQL but similar tables exist for other databases (see slides).

```
SELECT * 
FROM pg_catalog.pg_tables
WHERE schemaname = 'public';
```

# Determine the monthly income

Now that you know how to find the table that you need to answer a question and how to use SQL to answer that question let's practice these skills end-to-end.

How much does this business make per month?

First, you will need to use `pg_catalog.pg_tables` to find the possible tables and determine which tables & columns you need to answer that question. Second, you will leverage the tools you learned in the previous chapter to prepare the answer.

```
-- List all tables in the public schema
SELECT * 
FROM pg_catalog.pg_tables
WHERE schemaname = 'public';

-- Explore the tables and fill in the correct one
SELECT * 
FROM payment 
LIMIT 10;

-- Prepare the result
SELECT EXTRACT(MONTH FROM payment_date) AS month, 
       SUM(amount) AS total_payment
FROM payment 
GROUP BY month;
```

# What columns are in your database?

Just like `pg_catalog.pg_tables` can be incredibly helpful for listing all the tables in your database, `information_schema`.columns can be used to list the columns of these tables. In this exercise, you will combine these system tables to get a list of all of the columns for all your tables (in the `'public'` schema).

Note: These system tables are specific to PostgreSQL but similar tables exist for other databases (see slides).

```
SELECT table_name, column_name
FROM information_schema.columns
WHERE table_schema = 'public';
```

# A VIEW of all your columns

In this exercise you will create a new tool for finding the tables and columns you need. Using the system table `information_schema.columns` you will concatenate the list of each table's columns into a single entry.

Once you've done this you will make this query easily reusable by creating a new `VIEW` for it called `table_columns`.

```
-- Create a new view called table_columns
CREATE VIEW table_columns AS
SELECT table_name, 
	   STRING_AGG(column_name, ', ') AS columns
FROM information_schema.columns
WHERE table_schema = 'public'
GROUP BY table_name;

-- Query the newly created view table_columns
SELECT * FROM table_columns;
```

# Testing out your new VIEW

You are interested in calculating the average movie length for every category. Which tables & columns will you need to create this query?

Note: The `table_columns` view is now stored in your database and can be used to help you with this question.

- Tables: `film` & `category` -- Join on: `film_id`

# The average length of films by category

From the previous exercise you've learned that the tables `film` and `category` have the necessary information to calculate the average movie length for every category. You've also learned that they share a common field `film_id` which can be used to join these tables. Now you will use this information to query a list of average `length` for each `category`.

```
-- Calculate the average_length for each category
SELECT category, 
	   AVG(length) AS average_length
FROM film AS f
-- Join the tables film & category
INNER JOIN category AS c
  ON f.film_id = c.film_id
GROUP BY category
-- Sort the results in ascending order by length
ORDER BY average_length;
```

# Build the entity relationship diagram

Using the skills you learned throughout this chapter you will build an entity relationship diagram to trace and connect the tables needed to answer the following question:

Which films are most frequently rented?

- First explore `film`
- Then explore `rental`
- Then explore intermediate tables to connect them `inventory`
<center><img src="images/02.12.png"  style="width: 400px, height: 300px;"/></center>


# Which films are most frequently rented?

Now that you've figured out the relationships between the tables and their columns, you are ready to answer the question we started with:

Which films are most frequently rented?

Use the relationship diagram to answer this question.

<center><img src="images/02.13.png"  style="width: 400px, height: 300px;"/></center>


```
SELECT title, COUNT(title)
FROM film AS f
INNER JOIN inventory AS i
  ON f.film_id = i.film_id
INNER JOIN rental AS r
  ON i.inventory_id = r.inventory_id
GROUP BY title
ORDER BY count DESC;
```